## Regression Case


In [1]:
# libraries
import numpy as np

In [ ]:
# other test variables, which are self parameters
n_total_ftr = 0
target_values = [1,2,3,4,5,6,7,8,9,0]
mu = np.ones() * 0
sigma = np.ones() * 1
penalty_s = 0.01
penalty_r = 0.01
epochs = 1
lr_mu = 0.01
lr_sigma = 0.01
n_mc_samples = 5 #10000

# create 3d array with all r for current observation 
# r^cl_j = r[l, j, c]
#needs to be specified only if we got right model
# r = np.random.randn(monte_carlo, n_total_ftr)

# maybe set param amount_classes from target_values

In [ ]:
def __regression(x,y): # needs self in model
    """
        Update the distribution parameters mu and sigma by optimizing them in terms of the (log) likelihood.
        Here we assume a Bernoulli distributed target variable. We use a Probit model as our base model.
        This corresponds to the FIRES-GLM model in the paper.

        :param x: (np.ndarray) Batch of observations (numeric values only, consider normalizing data for better results)
        :param y: (np.ndarray) Batch of labels: type integer e.g. 1,2,3,4 usw
     """

     for epoch in range(epochs): #changed to self.epoch in model
         # Shuffle the observations
         n_obs = len(y) # problem if only one is given, handle later try catch or so
         random_idx = np.random.permutation(len(y))
         x = x[random_idx]
         y = y[random_idx]

         
         # Iterative update of mu and sigma
         try:
             # has shape o: observations, l: samples, j: features 
             r = np.random.randn(n_obs, n_mc_samples, n_total_ftr)
             print("R shape: {}".format(r.shape))
             # calculate thetas for all samples and observations
             theta = np.einsum("olj,j->olj",r,sigma) + mu
             
             # calculate marginal likelihood shape o
             marginal = np.einsum("olj,oj->olj",test,x) #theta *x
             marginal = np.einsum("olj->o", marginal) / n_mc_samples # sum over l and j / n_mc_samples
             print("Marginal shape: {}".format(marginal.shape))

             # calculate derivatives
             nabla_mu = x # shape oxj
             # 'ij->i' sum over all rows
             nabla_sigma = x * (np.einsum("olj->oj", r) / n_mc_samples) #shape oxj
            
             #update mu and sigma
             mu += lr_mu * np.mean(nabla_mu.T / marginal, axis=1)
             sigma += lr_sigma * np.mean(nabla_sigma.T / marginal, axis=1)


In [18]:
#test arrangment r, sigma mu
sigma = np.ones(5) * 2
mu = np.ones(5)
r = np.random.randn(5,5)
print(r)
r*sigma+mu

[[-0.2694132   1.74360548 -0.81616493 -0.95192273 -1.153926  ]
 [ 0.61750079 -1.50378716 -1.26957848  0.50592964  0.56013337]
 [ 1.91835193  0.17786443 -0.41306479  0.96489759  1.73160213]
 [-1.38518285 -1.15311577 -0.1431769   0.1605285  -0.59358681]
 [ 0.68371675 -0.34167393 -0.21781114 -1.31752758  0.83507045]]


array([[ 0.4611736 ,  4.48721095, -0.63232986, -0.90384547, -1.307852  ],
       [ 2.23500158, -2.00757433, -1.53915695,  2.01185928,  2.12026673],
       [ 4.83670385,  1.35572886,  0.17387042,  2.92979518,  4.46320427],
       [-1.7703657 , -1.30623154,  0.7136462 ,  1.321057  , -0.18717363],
       [ 2.3674335 ,  0.31665215,  0.56437771, -1.63505517,  2.67014089]])

In [17]:
# mlh calc
test1 = np.array([[1,2,3], [1,2,3], [1,2,3]])
print(test1)
np.einsum("ij->i", test1) # for nabla

[[1 2 3]
 [1 2 3]
 [1 2 3]]


array([6, 6, 6])

In [23]:
np.sum(test1, axis=0)

array([3, 6, 9])

In [25]:
np.einsum("ij->j", test1)

array([3, 6, 9])

In [22]:
test = np.ones((3,5,4))
print(test)
mu = np.array((4,3,2,1))
sigma = np.array((1,2,3,4))
np.einsum("olj,j->olj",test,sigma) + mu

[[[1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]

 [[1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]

 [[1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]]


array([[[5., 5., 5., 5.],
        [5., 5., 5., 5.],
        [5., 5., 5., 5.],
        [5., 5., 5., 5.],
        [5., 5., 5., 5.]],

       [[5., 5., 5., 5.],
        [5., 5., 5., 5.],
        [5., 5., 5., 5.],
        [5., 5., 5., 5.],
        [5., 5., 5., 5.]],

       [[5., 5., 5., 5.],
        [5., 5., 5., 5.],
        [5., 5., 5., 5.],
        [5., 5., 5., 5.],
        [5., 5., 5., 5.]]])

In [29]:
x = np.array((np.ones(4), np.ones(4)*2, np.ones(4)*3))
x
theta_x = np.einsum("olj,oj->olj",test,x)
np.einsum("olj->o", theta_x)

array([20., 40., 60.])

In [45]:
np.mean(np.einsum("olj->oj",theta_x).T / np.array((3,3,3)), axis=1)

array([3.33333333, 3.33333333, 3.33333333, 3.33333333])

In [46]:
np.einsum("olj->oj",theta_x).T 

array([[ 5., 10., 15.],
       [ 5., 10., 15.],
       [ 5., 10., 15.],
       [ 5., 10., 15.]])

In [57]:
np.mean(np.ones((3,4)).T / np.array((1,2,3)), axis=1)

array([0.61111111, 0.61111111, 0.61111111, 0.61111111])

In [52]:
test = np.ones((2,3,4))
print(test)
test = np.einsum("olj->o", test)
test

[[[1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]

 [[1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]]


array([12., 12.])